In [1]:
from __future__ import division
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

from pybasicbayes.distributions import Regression
from pybasicbayes.util.text import progprint_xrange
from autoregressive.distributions import AutoRegression



import pyximport
pyximport.install()
import os
os.chdir('../pylds')
from models import LDS, DefaultLDS
from distributions import Regression_diag, AutoRegression_input

npr.seed(0)

def update(model):
    model.EM_step()
    return model.log_likelihood()                    

def init_LDS_model(pars,data):
    model = LDS(dynamics_distn=AutoRegression_input(
                  nu_0=n+1, S_0=n*np.eye(n), M_0=np.zeros((n, n)), K_0=n*np.eye(n),
                  A=pars['A'].copy(), sigma=pars['Q'].copy()),                
                emission_distn=Regression_diag(
                  nu_0=p+1, S_0=p*np.eye(p), M_0=np.zeros((p, n+1)), K_0=p*np.eye(n+1),
                  A=np.hstack((pars['C'].copy(), pars['d'].copy().reshape(p,1))), 
                  sigma=pars['R'].copy(),
                  affine=True))
    
    model.mu_init = pars['mu0'].copy()
    model.sigma_init = pars['V0'].copy()
    
    model.add_data(data)
    return model

def collect_LDS_stats(model):
    stats =  {'mu_h' : model.states_list[0].smoothed_mus.copy(),
              'V_h'  : model.states_list[0].smoothed_sigmas.copy(),
              'yy': model.states_list[0].E_emission_stats[0],
              'yx': model.states_list[0].E_emission_stats[1],
              'xx': model.states_list[0].E_emission_stats[2],
              'extxtm1': model.states_list[0].E_addition_stats.copy()}
    pars = {'A' : model.A.copy(),
            'B' : 0,
            'Q' : model.sigma_states.copy(),
            'mu0': model.mu_init.copy(),
            'V0': model.sigma_init.copy(),
            'C' : model.C.copy(),
            'd' : model.d.copy(),
            'R' : model.sigma_obs.copy()}    
    return stats, pars

#########################
#  set some parameters  #
#########################

n = 3
p = 9
T = 10000

A  = np.diag(np.linspace(0.90, 0.98, n)) 
while True:
    W    = np.random.normal(size=[n,n])
    if np.abs(np.linalg.det(W)) > 0.001:
        break
pars_true = {'A': A.copy(),
             'Q': 0.01*np.eye(n),
             'B': 0,
             'mu0': 1.0 * np.arange(n),
             'V0': 0.01*np.eye(n),
             'C': np.random.normal(size=(p,n)),
             'd' : np.random.normal(size=(p)),
             'R': 0.01*np.ones(p).reshape(p,)}

A  = 0.99 * np.diag(np.ones(n)) 
while True:
    W    = np.random.normal(size=[n,n])
    if np.abs(np.linalg.det(W)) > 0.001:
        break
pars_init = {'A' : A.copy(),
             'B' : 0,
             'Q' : np.eye(n),
             'mu0': 0.9 * np.arange(n),
             'V0': 0.01*np.eye(n),
             'C' : np.random.randn(p,n),
             'd' : np.random.normal(size=(p)),
             'R' : 0.1*np.ones(p)}


###################
#  generate data  #
###################

truemodel = LDS(
    dynamics_distn=AutoRegression(A=pars_true['A'].copy(),sigma=pars_true['Q'].copy()),
    emission_distn=Regression_diag(A=np.hstack((pars_true['C'].copy(), pars_true['d'].copy().reshape(p,1))),
                                   sigma=pars_true['R'].copy(), affine=True),
                )
truemodel.mu_init = pars_true['mu0'].copy()
truemodel.sigma_init = pars_true['V0'].copy()

data, stateseq = truemodel.generate(T)


# state after learning with E_step(), i.e. full-matrix version

# get E-step results for init pars
model = init_LDS_model(pars_init, data) # set to initialisation
model.E_step()
stats_init,_ = collect_LDS_stats(model)

# get EM-step results after one iteration
model.M_step()
model.E_step()
stats_first,pars_first = collect_LDS_stats(model)

# get EM-step results after 50 iterations                    
model = init_LDS_model(pars_init, data) # reset to initialisation                    
print 'fitting'
likes = [update(model) for _ in progprint_xrange(2)]
stats_hat,pars_hat = collect_LDS_stats(model)
                                                        
# get EM-step results from true parameters
model = init_LDS_model(pars_true, data) # reset to true pars
model.E_step()
stats_true,_ = collect_LDS_stats(model)
model.M_step()
                    
                    

save_file = '../../../results/pylds_debug/' + 'pylds_1x2_diag'
from scipy.io import savemat # store results for comparison with Matlab code   
from scipy.linalg import solve_discrete_lyapunov as dtlyap # solve discrete-time Lyapunov equation
save_file_m = {'x': model.states_list[0].stateseq, 
               'y': model.states_list[0].data,
               'u' : [], 
               'll' : likes, 
               'T' : model.states_list[0].T, 
               'Trial': len(model.states_list), 
               'elapsedTime' : 0,
               'ifUseB':False, 
               'ifUseA':True, 
               'epsilon':0,
               'ifRDiagonal':False,
               'covConvEps':0,        
               'truePars':pars_true,
               'initPars':pars_init,
               'firstPars':pars_first,
               'estPars': pars_hat,
               'stats_0': stats_init,
               'stats_1': stats_first,
               'stats_h': stats_hat,
               'stats_true': stats_true,
               'Pi':dtlyap(pars_true['A'], pars_true['Q']),
               'Pi_h':dtlyap(pars_hat['A'], pars_hat['Q']),
               'Pi_t':pars_true['A'].dot(dtlyap(pars_true['A'], pars_true['Q'])),
               'Pi_t_h':pars_hat['A'].dot(dtlyap(pars_hat['A'], pars_hat['Q'])),
               'obsScheme' : []}

savemat(save_file,save_file_m) # does the actual saving


('yyT.shape', (9,))
('dsigma.shape', (9,))
('sigma.shape', (9, 9))
fitting
('yyT.shape', (9,))
('dsigma.shape', (9,))
('sigma.shape', (9, 9))
.('yyT.shape', (9,))
('dsigma.shape', (9,))
('sigma.shape', (9, 9))
.
   0.26sec avg, 0.52 total

('yyT.shape', (9,))
('dsigma.shape', (9,))
('sigma.shape', (9, 9))
